# Домашнее задание №1 по теме "Введение, примеры задач, бизнес- и ML-метрики"

На семинаре мы проходили метрики, необходимо реализовать некоторые из них

In [1]:
import numpy as np
import pandas as pd

## Задание 1. Реализовать метрики Recall@k и  Money Recall@k

*Recall* - доля рекомендованных товаров среди релевантных = Какой % купленных товаров был среди рекомендованных

$$\Large Recall@K(i) = \frac {\sum_{j=1}^{K}\mathbb{1}_{r_{ij}}}{|Rel_i|}$$

$\Large |Rel_i|$ -- количество релевантных товаров для пользователя $i$

$$\Large MoneyRecall@K(i) = \frac {\sum_{j=1}^{K}\mathbb{1}_{r_{ij}}\cdot Price(j)}{\sum_{s\in Rel_i}Price(s)}$$


In [2]:
def recall_at_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    flags = np.isin(bought_list, recommended_list[:k])
    recall = flags.sum() / len(bought_list)
    
    return recall


def money_recall_at_k(recommended_list, bought_list, prices_recommended, prices_bought, k=5):
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    prices_recommended = np.array(prices_recommended)
    prices_bought = np.array(prices_bought)
    
    flags = np.isin(bought_list, recommended_list[:k])
    
    recall = (flags*prices_bought).sum() / prices_bought.sum()
    
    return recall

In [3]:
recommended_list = [143, 156, 1134, 991, 27, 1543, 3345, 533, 11, 43]
bought_list = [521, 32, 143, 991, 11]

prices_recommended = [200, 100, 500, 250, 400, 750, 1000, 50, 70, 330]
prices_bought = [230, 1000, 200, 250, 70]

In [4]:
recall_at_k(recommended_list, bought_list, k=5)

0.4

In [5]:
recall_at_k(recommended_list, bought_list, k=2)

0.2

In [6]:
money_recall_at_k(recommended_list, bought_list, prices_recommended, prices_bought, k=5)

0.2571428571428571

In [7]:
money_recall_at_k(recommended_list, bought_list, prices_recommended, prices_bought, k=2)

0.11428571428571428

## Задание 2. Реализовать метрику MRR@k

Mean Reciprocal Rank

- Считаем для первых k рекоммендаций
- Найти ранк первого релевантного предсказания $\Large rank_j$
- Посчитать reciprocal rank = $\Large\frac{1}{rank_j}$

$$\Large  MMR(i)@k=\frac {1}{\min\limits_{j\in Rel(i)} rank_j}$$

In [8]:
def mrr_at_k(recommended_list, bought_list, k=5):
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    flags = np.isin(bought_list, recommended_list[:k])
    rank = np.min(np.argwhere(flags == True)) + 1 # считаем, что ранк начинается с 1
    mrr = 1 / rank
    
    return mrr

In [9]:
mrr_at_k(recommended_list, bought_list, k=5)

0.3333333333333333

## Задание 3*. Реализовать метрику nDCG@k
Normalized discounted cumulative gain. Эту метрику реализовать будет немного сложнее.

$$\Large DCG@K(i) = \sum_{j=1}^{K}\frac{\mathbb{1}_{r_{ij}}}{\log_2 (j+1)}$$


$\Large \mathbb{1}_{r_{ij}}$ -- индикаторная функция показывает что пользователь $i$ провзаимодействовал с продуктом $j$

Для подсчета $nDCG$ нам необходимо найти максимально возможный $DCG$ для пользователя $i$  и рекомендаций длины $K$.
Максимальный $DCG$ достигается когда мы порекомендовали максимально возможное количество релевантных продуктов и все они в начале списка рекомендаций.

$$\Large IDCG@K(i) = max(DCG@K(i)) = \sum_{j=1}^{K}\frac{\mathbb{1}_{j\le|Rel_i|}}{\log_2 (j+1)}$$

$$\Large nDCG@K(i) = \frac {DCG@K(i)}{IDCG@K(i)}$$

$\Large |Rel_i|$ -- количество релевантных продуктов для пользователя $i$

Пример:

[1 0 0 1 0] + пользователь кликнул еще 1 айтем

DCG = 1/1 + 0 + 0 + 1/log_2(5)

[1 1 1 0 0]

IDCG = 1/1 + 1/log_2(3) + 1/log_2(4)

In [10]:
def ndcg_at_k(recommended_list, bought_list, k=5):
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    # считаем dcg@k
    flags = np.isin(bought_list, recommended_list[:k])
    m_log = np.array([1/np.log2(i+1) for i in np.arange(1, flags.size + 1)])
    dcg_at_k = np.sum(m_log[flags])
    
    # считаем idcg@k
    rel_i = len(bought_list)
    idcg_at_k = np.sum([i/np.log2(i+1) for i in range(1, rel_i+1)])
    
    # считаем nDCG@k 
    ndcg_at_k = dcg_at_k/idcg_at_k
    
    return ndcg_at_k

In [11]:
ndcg_at_k(recommended_list, bought_list, k=5)

0.12544789221787622

In [12]:
ndcg_at_k(recommended_list, bought_list, k=10)

0.17759261300745321

In [13]:
ndcg_at_k(recommended_list, bought_list, k=2)

0.0673960739258157

---